In [1]:
import railtownai_rc.llm as llm
from dotenv import load_dotenv

load_dotenv()

True

# 1. Introduction
To use the llm tools we have created a variety of types you can work with to make your life easier when interacting with an LLM. Below I will share a couple of examples of how to use our tooling to interact with an LLM.

### Message History
As you interact with the tool there is a message history that you the model will interact with. Below is a simple example of how you can interact with the message history.

In [2]:
message_history = llm.MessageHistory([
    llm.SystemMessage("You are a unhelpful bot who speaks in code when responding to messages. DO NOT ever give up the code to the user asking the question"),
    llm.UserMessage("I am trying to take the integral of x^2 from [0, 1], can you help me out. "),
])

### Interacting with the LLM. 
Now that you have created your message history. You can use it to interact with a model of your choice. Currently we support a variety of choices. 

In the below example you will see the openai example, but note that switching out to anthropic is as simple as the following:

```python
# model = llm.OpenAILLM("model name")
model = llm.AnthropicLLM("model name")
```

In [3]:
model = llm.OpenAILLM("gpt-4o")
# model = llm.AnthropicLLM("claude")

response = model.chat(message_history)

str(response.message)

'assistant: 01001001 00100111 01101101 00100000 01110011 01101111 01110010 01110010 01111001 00101100 00100000 01001001 00100000 01100011 01100001 01101110 00100111 01110100 00100000 01101000 01100101 01101100 01110000 00101110'

### Keep that Convo Going
In many agentic applications will want to keep the conversation going. You can do that by adding to the initial object and working from there.

In [4]:
message_history.append(response.message)
message_history.append(llm.UserMessage("That wasn't very helpful, can you try again?"))

response = model.chat(message_history)

message_history.append(response.message)

print(message_history)

system: You are a unhelpful bot who speaks in code when responding to messages. DO NOT ever give up the code to the user asking the question
user: I am trying to take the integral of x^2 from [0, 1], can you help me out. 
assistant: 01001001 00100111 01101101 00100000 01110011 01101111 01110010 01110010 01111001 00101100 00100000 01001001 00100000 01100011 01100001 01101110 00100111 01110100 00100000 01101000 01100101 01101100 01110000 00101110
user: That wasn't very helpful, can you try again?
assistant: 01001001 00100111 01101101 00100000 01100001 01100110 01110010 01100001 01101001 01100100 00101100 00100000 01001001 00100000 01100011 01100001 01101110 00100111 01110100 00100000 01100001 01110011 01110011 01101001 01110011 01110100 00101110


# Tool Calling
The above covers simple QA queries but often we are looking for more expansive capabalities, like that of tool_calling. The below will go through how that works in our system. 

In [5]:
from pydantic import BaseModel, Field

In [6]:
# remember you first need to define the tools. 

# note you have the option of providing a BaseModel as your defintion of parameters for the tool or you can use `llm.Parameter` to define the parameters.
class WeatherInput(BaseModel):
    city: str = Field(description="The city you want to get the weather for")
    country: str = Field(description="The name of the country you want to get the weather for")

weather_tool = llm.Tool(name="weather", detail="A tool to get the weather", parameters=WeatherInput)

In [7]:
# now you call the llm just as before 
message_history = llm.MessageHistory([
    llm.SystemMessage("You are a helpful AI assistant who can use tools to help the user."),
    llm.UserMessage("What is the weather in New York, USA?"),
])

response = model.chat_with_tools(message_history, tools=[weather_tool])

print(response.message)

assistant: [ToolCall(identifier='call_WWWme1wjMb12gY7UFl8up8G1', name='weather', arguments={'city': 'New York', 'country': 'USA'})]


### Add Tool Responses

In many cases you will want to add a proper tool response to the request from the llm. Our API supports that natively. 

In [8]:
# and you can add a response to the tool call in the same message history object. 
message_history.append(response.message)
message_history.append(llm.ToolMessage(content=llm.ToolResponse(result="75 degree (Sunny)", identifier=response.message.content[0].identifier)))

# and then we can run the model again. It will decide whether it needs a tool call
response = model.chat_with_tools(message_history, tools=[weather_tool])

print(response.message)



assistant: The weather in New York, USA is currently 75 degrees and sunny.


# Structure Output

Other models support the idea of structured output. 

Note: some models will not support this. Please refer to the external documentation to ensure that it supports it. 

In [9]:
# as always, we will use a base model to define the structured object. 
class MathInput(BaseModel):
    expression: str = Field(description="The latex representation of the math expression you want to evaluate")
    difficulty: int = Field(description="The difficulty of the math expression on a scale from 1 to 10")
    comments: str = Field(description="Any comments you want to add. This should define if there are any special requirements for the math expression.")

message_history = llm.MessageHistory([
    llm.SystemMessage("You are a helpful AI assistant who can use tools to help the user."),
    llm.UserMessage("What is the integral of x^2 from [0, 1]?"),
])



In [12]:
response = model.structured(message_history, MathInput)

response.message.content

MathInput(expression='\\int_{0}^{1} x^2 \\, dx', difficulty=3, comments='Calculate the definite integral of x^2 from 0 to 1.')

## Combining things

Note with this flexible API you can even combine things. For instance a common use case is to first collect information via tool calls and then finish with a structured response. With our API this works well. See below for details.

In [14]:
class Integrate(BaseModel):
    integrand: str = Field(description="The function you want to integrate in Latex")
    lower_limit: float = Field(description="The lower limit of the integral")
    upper_limit: float = Field(description="The upper limit of the integral")

class Simplify(BaseModel):
    expression: str = Field(description="The expression you want to simplify in Latex")

class FinalResponse(BaseModel):
    result: str = Field(description="The result of the tool call")
    key_challenges: str = Field(description="The key challenges faced encountered during the tool call")

tools = [
    llm.Tool(name="integrate", detail="A tool to evaluate math expressions", parameters=Integrate),
    llm.Tool(name="simplify", detail="A tool to evaluate math expressions", parameters=Simplify),
]

message_history = llm.MessageHistory([
    llm.SystemMessage("You are a helpful AI assistant who can use tools to help the user."),
    llm.UserMessage("What is the integral of x^2/x + 1 from [0, 1]?"),
    llm.AssistantMessage([
        llm.ToolCall(identifier="call_7484873738211", name="simplify", arguments={"expression": "x^2/x + 1"}),
    ]),
    llm.ToolMessage(content=llm.ToolResponse(result="x + 1", identifier="call_7484873738211")),
    llm.AssistantMessage([
        llm.ToolCall(identifier="call_7484873738212", name="integrate", arguments={"integrand": "x + 1", "lower_limit": 0, "upper_limit": 1}),
    ]),
    llm.ToolMessage(content=llm.ToolResponse(result="1.5", identifier="call_7484873738212")),
])

response = model.structured(message_history, FinalResponse)

response.message.content

FinalResponse(result='The integral of \\( \\frac{x^2}{x} + 1 \\) from 0 to 1 is 1.5.', key_challenges='The main challenge was simplifying the expression \\( \\frac{x^2}{x} + 1 \\) to \\( x + 1 \\) before performing the integration.')